In [4]:
import pandas as pd
import ast
import json
import re
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import warnings
from scipy.stats import pearsonr
warnings.filterwarnings('ignore')
from itertools import combinations
import random
from seqeval.metrics import f1_score,classification_report,accuracy_score
from sklearn.metrics import f1_score as bi_f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score as bi_accuracy_score
from sklearn.metrics import classification_report as bi_classification_report
import traceback
import html #for unescape & < >
from scipy import stats
import matplotlib.pyplot as plt
from collections import defaultdict
import emoji
import os


In [5]:
import traceback


all_gs_sub=pd.read_json('/Users/ruofanhu/Desktop/submission/data/expert:gold_standard_sub/gold_standard_500.json')

csvfile_500='/Users/ruofanhu/Desktop/submission/data/expert:gold_standard_sub/expert_label_500.csv'
data_i=pd.read_csv(csvfile_500)

# set tweet_id+worker_id as index and group by tweet_id 
ppdict_i = {n: grp.loc[n].to_dict('index') for n, grp in data_i.set_index(['new_tweet_id', 'WorkerId']).groupby(level='new_tweet_id')}

new_dict_i={}
for tweet_id,collection in ppdict_i.items():
    basic_info=list(collection.values())[0]
    # seperate strings but keep delimiters
    txt = html.unescape(basic_info['Input.tweet'])
    
    temp = list(filter(None, re.split('([,.!?:()[\]"\s+])', txt)))

    # remove space strings from list and convert into np array
    tweet_split = np.array(list(filter(str.strip, temp)))
    #     tweet_split = np.array(re.split('[,.!?:()[\]"\s+]', basic_info['Input.tweet']))
#     gold_standard = basic_info['Input.prediction']
    workerids = list(collection.keys())
    
    sentence_score = []
    entity_label = []
    related_label = []
    workertime = []
    assignmentID = []
    for workerid,record in collection.items():
        sentence_score.append(record['Answer.optradio'])
        workertime.append(record['WorkTimeInSeconds'])
        token_labels = np.array(['O']*len(tweet_split),dtype=np.dtype(('U',10)))
        relation_lables = np.array([0]*len(tweet_split))
        assignmentID.append(record['AssignmentId'])
        try:
            if record['Answer.related_index'] != '[]' :
    #             print(record['Answer.related_index'])
    #             print('undefined' in record['Answer.related_index'])
                relation_lables_idx_str = sum([i.split(' ') for i in ast.literal_eval(record['Answer.related_index'])],[])

                relation_lables_idx = list(map(int, relation_lables_idx_str))
                relation_lables[relation_lables_idx] = 1

                
            if np.isnan(record['Answer.no-entity']) and re.split('[|]', record["Answer.html_output"])[1]!='': # the value is 1 when there is no entity to label
                html_output_list = ast.literal_eval(re.split('[|]', record["Answer.html_output"])[1])
                
                for e in html_output_list:
                    if 'idx' in list(e.keys()):

                        if ' ' in e['idx']:
                            idx = list(map(int, e['idx'].split(' ')))
                        else:
                            idx = ast.literal_eval(e['idx'])

                        if type(idx) is int:

                            assert tweet_split[idx] == e['text']
                            token_labels[idx] = 'B-'+e['className'].split('-')[1]
                        else:
    #                         print(' '.join(tweet_split[idx]))
    #                         print(e['text'])
#                             if tweet_split[idx][0] != e['text'].split()[0] and  tweet_split[idx][-1] != e['text'].split()[-1]:
#                                 print(tweet_split[idx],e['text'])
                            assert tweet_split[idx][0] == e['text'].split()[0] and  tweet_split[idx][-1] == e['text'].split()[-1]
                            idx=list(idx)
                            token_labels[idx[0]] = 'B-'+e['className'].split('-')[1]
                            token_labels[idx[1:]] = 'I-' + e['className'].split('-')[1]


                
        except Exception:
            traceback.print_exc()
            print('AssignmentId:',record['AssignmentId'],'Answer.related_index:',record['Answer.related_index'])

        entity_label.append(token_labels.tolist())
        related_label.append(relation_lables.tolist())
    new_dict_i[tweet_id]={'tweet':txt,'tweet_tokens':tweet_split.tolist(),
                       'workerid':workerids,'workertime':workertime,'sentence_score':sentence_score,
                        'entity_label':entity_label,'related_label':related_label,'assignmentID':assignmentID}




In [6]:
g=pd.DataFrame.from_dict(new_dict_i)
gg=g.T

In [7]:

gg['sentence_class']=gg['sentence_score'].apply(lambda x: int(x[0]>3))

In [8]:
all_gs_sub=all_gs_sub.sort_values(by ='new_tweet_id' )
new_gg=gg.sort_index()

In [9]:

gt_sc=new_gg['sentence_class'].tolist()
crowd_sc=all_gs_sub['sentence_class'].tolist()
# crowd_sc=all_cd_expert_labeled_data['sen_mean_t'].tolist()


print('F1:',bi_f1_score(gt_sc,crowd_sc),
      '\nACC:',bi_accuracy_score(gt_sc,crowd_sc),
      '\nAUC:',roc_auc_score(gt_sc,crowd_sc))

F1: 0.8189806678383128 
ACC: 0.794 
AUC: 0.802472897555969


# create mat for krippendorff's alpha

In [ ]:
data=pd.read_csv('expert_label_500.csv')


dj=data.to_json(orient="records")
parsed = json.loads(dj)

def process_entity_label(record):
    txt = html.unescape(record['Input.tweet'])
    
    temp = list(filter(None, re.split('([,.!?:()[\]"\s+])', txt)))

    # remove space strings from list and convert into np array
    tweet_split = np.array(list(filter(str.strip, temp)))


    token_labels = np.array(['']*len(tweet_split),dtype=np.dtype(('U',10)))
#     if record['Answer.related_index'] != '[]' :
#         relation_lables_idx_str = sum([i.split(' ') for i in ast.literal_eval(record['Answer.related_index'])],[])

#         relation_lables_idx = list(map(int, relation_lables_idx_str))
#         relation_lables[relation_lables_idx] = 1

                
    if record['Answer.no-entity'] is None and re.split('[|]', record["Answer.html_output"])[1]!='': # the value is 1 when there is no entity to label
        html_output_list = ast.literal_eval(re.split('[|]', record["Answer.html_output"])[1])
                
        for e in html_output_list:
            if 'idx' in list(e.keys()):

                if ' ' in e['idx']:
                    idx = list(map(int, e['idx'].split(' ')))
                else:
                    idx = ast.literal_eval(e['idx'])

                if type(idx) is int:

#                             assert tweet_split[idx] == e['text']
                    token_labels[idx] = 'B-'+e['className'].split('-')[1]
                else:
                    idx=list(idx)
                    token_labels[idx[0]] = 'B-'+e['className'].split('-')[1]
                    token_labels[idx[1:]] = 'I-' + e['className'].split('-')[1]



    return token_labels.tolist()

In [ ]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
reviewsPertweet = defaultdict(list)
for d in parsed:
    entity_labels=process_entity_label(d)


    for i in range(len(entity_labels)):
        user,item = 'expert', str(d['new_tweet_id'])+','+str(i)
        
        reviewsPerItem[item].append({'WorkerId':user,'entity_type':entity_labels[i]})



In [ ]:
# add gold_standard label
input_id=all_gs_sub.index
entity_labels_l=all_gs_sub['entity_label'].tolist()
for i in range(all_gs_sub.shape[0]):
    entity_labels=entity_labels_l[i]
    
    
    for j in range(len(entity_labels)):
        user,item = 'gold_standard', str(input_id[i])+','+str(j)

        reviewsPerItem[item].append({'WorkerId':user,'entity_type':entity_labels[j]})

lu=2
li = len(reviewsPerItem)
# data_ = np.empty((li, lu))
# data_[:] = np.nan
# data__t = np.empty((lu, li))
ku = ['expert','gold_standard']
ki = list(reviewsPerItem.keys())

# for i in range(li):
#     for r in reviewsPerItem[ki[i]]:
#         data_[i][ku.index(r['WorkerId'])] =r['entity_type']

# Construct the P Matrix
data_m = pd.DataFrame('',columns=ku,index=ki)
for i in range(li):
    for r in reviewsPerItem[ki[i]]:
        if r['entity_type']!='O' and r['entity_type']!='' :
            
            data_m.loc[ki[i]][ku.index(r['WorkerId'])] = r['entity_type'].split('-')[1]
        
        else:
        
            data_m.loc[ki[i]][ku.index(r['WorkerId'])] = r['entity_type']




In [ ]:
new_data_m=data_m.replace('O','')
new_data_m.to_csv('input_to_software.csv',index=False)